In [9]:
import sqlite3

In [10]:
# Connect to the SQLite database
conn = sqlite3.connect('european_database.sqlite')

In [11]:
cursor = conn.cursor()
# Check for missing values in 'FTHG' and 'FTAG' columns
cursor.execute("SELECT COUNT(*) FROM matchs WHERE FTHG IS NULL OR FTAG IS NULL")
missing_values_count = cursor.fetchone()[0]

if missing_values_count > 0:
    # Handle missing values as needed (e.g., imputation)
    # Example: Impute missing values with zeros
    cursor.execute("UPDATE matchs SET FTHG = 0 WHERE FTHG IS NULL")
    cursor.execute("UPDATE matchs SET FTAG = 0 WHERE FTAG IS NULL")
    conn.commit()

In [12]:
# Ensure that 'FTHG' and 'FTAG' columns contain valid numerical values
cursor.execute("PRAGMA foreign_keys = ON;")
cursor.execute("PRAGMA table_info(matchs)")

columns = cursor.fetchall()
for column in columns:
    if column[1] in ['FTHG', 'FTAG'] and column[2] != 'REAL':
        # If the column is not of type REAL, convert it to REAL
        cursor.execute(f"ALTER TABLE matchs RENAME TO matchs_old;")
        cursor.execute(f"CREATE TABLE matchs AS SELECT * FROM matchs_old;")
        cursor.execute(f"DROP TABLE matchs_old;")
        conn.commit()

In [17]:
# Select goals scored in home matches
cursor.execute("SELECT FTHG FROM matchs")
home_goals = [row[0] for row in cursor.fetchall()]

# Select goals scored in away matches
cursor.execute("SELECT FTAG FROM matchs")
away_goals = [row[0] for row in cursor.fetchall()]

# Close the database connection
conn.close()

In [18]:
import scipy.stats as stats

# Perform the t-test
t_stat, p_value = stats.ttest_ind(home_goals, away_goals)

# Set the significance level (alpha)
alpha = 0.05

# Check the p-value against alpha
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in goal statistics between home and away matches.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in goal statistics between home and away matches.")

Reject the null hypothesis: There is a significant difference in goal statistics between home and away matches.
